### 블록체인프로그래밍 13주차 과제
##### 휴먼지능정보공학과 201810756 김대환 

---
A, B 2인의 가위바위보 후 내기금액을 이긴 사람에게 지급하는 게임을 블록체인에 개발하세요.

게임 컨트랙은 Rsp, 구현할 함수는 다음과 같다. 그 외 필요로 하는 함수는 추가할 수 있다.

---

##### [컨트랙 설명]
* User struct 선언 : 이름, 배팅액, 자신의 패, 잔고를 가지고 있음
* 처음에 initBal() function을 통해 개인 잔고 충전(onlyOwner modifier 사용 -> 게임의 주인? 느낌, host가 허락해야 충전할 수 있도록)
* setA() function은 파라미터로 배팅액(uint)을 받음. 랜덤하게 0,1,2 난수 발생해서 자신의 패 결정
    * onlyAhasMoney modifier 사용 -> A가 자신의 잔액보다 높은 배팅액을 걸었을 경우 게임 실행 X
* setB() function은 파라미터로 패(uint), 배팅액(uint)를 받음. 
    * onlyBhasMoney modifier 사용 -> B가 자신의 잔액보다 높은 배팅액을 걸었을 경우 게임 실행 X
* play() function :  게임 실행 결과를 저장 
* distribuateBetAmount() function : 게임 실행 결과에 맞게 잔액 재설정
* getMatchResult() function : 게임 결과 반환 
* getBalance() function : 각 계정 및 컨트랙의 잔고 반환

In [2]:
%%writefile src/Rsp.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity 0.8.12;

contract RSP{
    struct User{
        string userName;
        uint amount;
        uint card;
        uint balance;
    }
    User A;
    User B;
    string result;
    address owner;
    constructor(){
        owner = msg.sender;
    }
    function initBal(uint _balance) payable public onlyOwner{
        require(msg.value == _balance);
        A.balance = _balance;
        B.balance = _balance;
    }
    function setA(uint _amount) public onlyAhasMoney{
        uint cardNum = uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3);
        // 0 : 가위, 1 : 바위, 2 : 보
        A.userName = "A";
        A.amount = _amount;
        A.card = cardNum;
    }
    function setB(uint _cardNum, uint _amount) public onlyBhasMoney{
        B.userName = "B";
        B.amount = _amount;
        B.card = _cardNum;
    }
    function play() public{
        uint cardA = A.card;
        uint cardB = B.card;
        if(cardA == cardB) result = "tie";
        else if((cardA == 0 && cardB == 2) || (cardA == 1 && cardB == 0) || (cardA == 2 && cardB ==1)) result = "A wins";
        else result = "B wins";
        }
    function distribuateBetAmount() payable public{
        if(keccak256(abi.encodePacked(result)) == keccak256(abi.encodePacked("A wins"))){
            A.balance += B.amount;
            B.balance -= B.amount;
        }
        else if(keccak256(abi.encodePacked(result)) == keccak256(abi.encodePacked("B wins"))){
            B.balance += A.amount;
            A.balance -= A.amount;
        }
    }
    function getMatchResult() public view returns (string memory){
        return result;
    }
    function getBalance() public view returns (uint, uint, uint){
        return (A.balance, B.balance, address(this).balance);
    }
    modifier onlyOwner(){
        require(msg.sender == owner);
        _;
    }
    modifier onlyAhasMoney(){
        require(A.balance >= A.amount);
        _;
    }
    modifier onlyBhasMoney(){
        require(B.balance >= B.amount);
        _;
    }
}


Overwriting src/Rsp.sol


In [3]:
!solc --optimize --combined-json abi,bin src/Rsp.sol > src/Rsp.json

In [4]:
!cat src/Rsp.json

{"contracts":{"src/Rsp.sol:RSP":{"abi":[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"distribuateBetAmount","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getMatchResult","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_balance","type":"uint256"}],"name":"initBal","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"play","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"setA","outputs":[],"stateMutability":"nonpayable","type":"function"},{"in

##### [배포하기]

In [7]:
%%writefile src/RspDeploy.js
var Web3 = require('web3');
var _abiBinJson = require('./Rsp.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
_bin=_abiBinJson.contracts[contractName].bin;
async function deploy(){
    const accounts = await web3.eth.getAccounts();
     new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> estimated gas: "+ gas);
    });
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data : "0x"+_bin})
        .send({from: accounts[0], gas: 482645}, function(err,transactionHash){
            if(!err) console.log("hash: "+transactionHash);
        })
    console.log(deployed.options.address)
}
deploy()

Overwriting src/RspDeploy.js


In [8]:
!node src/RspDeploy.js

>> estimated gas: 482645
hash: 0x69183ab06a838c176829dad66104923e67d5e2dd26d68a35931d0b6eb71b54ad
0x4c362EbF5142850e2EB9ff27F28101D7CBFbBFe7


##### [사용하기]

In [9]:
%%writefile src/RspUse_1.js
var Web3 = require('web3');
var _abiBinJson = require('./Rsp.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
var RspGame = new web3.eth.Contract(_abiArray,'0x4c362EbF5142850e2EB9ff27F28101D7CBFbBFe7');
async function use(){
    const accounts = await web3.eth.getAccounts();
    await RspGame.methods.initBal(3000).send({from:accounts[0], gas:259210, value : 3000});
    await RspGame.methods.getBalance().call().then(function(value){
        console.log("(1) [initial state of balance]\n"+"A: ",value[0]+"\nB: ",value[1]+"\nContract: ",value[2])
    });
}
use()

Writing src/RspUse_1.js


##### [A,B 플레이어 모두 초기 자금으로 3000 wei 충전]
##### [게임 전의 A잔고, B잔고, 컨트랙 잔고 출력]

In [10]:
!node src/RspUse_1.js

(1) [initial state of balance]
A:  3000
B:  3000
Contract:  3000


In [13]:
%%writefile src/RspUse_2.js
var Web3 = require('web3');
var _abiBinJson = require('./Rsp.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
var RspGame = new web3.eth.Contract(_abiArray,'0x4c362EbF5142850e2EB9ff27F28101D7CBFbBFe7');
async function use(){
    const accounts = await web3.eth.getAccounts();
    // A,B 모두 1000wei 배팅, B는 1(바위) 내기로 함
    await RspGame.methods.setA(1000).send({from:accounts[0], gas:259210});
    await RspGame.methods.setB(1,1000).send({from:accounts[0], gas:259210});
    await RspGame.methods.play().send({from:accounts[0],gas:259210});
    await RspGame.methods.getMatchResult().call().then(function(value){
        console.log("(2) [game result] : ", value)
    });
    await RspGame.methods.distribuateBetAmount().send({from:accounts[0], gas:259210});
    await RspGame.methods.getBalance().call().then(function(value){
        console.log("(3) [state of balance after Game]\n"+"A: ",value[0]+"\nB: ",value[1]+"\nContract: ",value[2])
    });
}
use()

Overwriting src/RspUse_2.js


#### [첫번째 경기, B가 바위를 냈고 A는 랜덤으로 결정, B가 이겼으므로 B의 잔고가 A의 배팅액 만큼 늘었음]

In [14]:
!node src/RspUse_2.js

(2) [game result] :  B wins
(3) [state of balance after Game]
A:  2000
B:  4000
Contract:  3000


##### [두번째 경기, 다시 B가 이김]

In [15]:
!node src/RspUse_2.js

(2) [game result] :  B wins
(3) [state of balance after Game]
A:  1000
B:  5000
Contract:  3000


##### [3번째 경기, 무승부, 두 플레이어 모두 잔고 변화 없음]

In [16]:
!node src/RspUse_2.js

(2) [game result] :  tie
(3) [state of balance after Game]
A:  1000
B:  5000
Contract:  3000


##### [4번째 경기, B의 승리, A의 잔고 0]

In [17]:
!node src/RspUse_2.js

(2) [game result] :  B wins
(3) [state of balance after Game]
A:  0
B:  6000
Contract:  3000


##### [A가 잔고가 없는 상태로 게임을 진행하려하자 예외 발생 -> 게임 실행 X]

In [18]:
!node src/RspUse_2.js

/Users/kimdaehwan/Desktop/Code/201810756/node_modules/web3-core-helpers/src/errors.js:29
        return new Error('Returned error: ' + message);
               ^

Error: Returned error: VM Exception while processing transaction: revert
    at Object.ErrorResponse (/Users/kimdaehwan/Desktop/Code/201810756/node_modules/web3-core-helpers/src/errors.js:29:16)
    at /Users/kimdaehwan/Desktop/Code/201810756/node_modules/web3-core-requestmanager/src/index.js:140:36
    at XMLHttpRequest.request.onreadystatechange (/Users/kimdaehwan/Desktop/Code/201810756/node_modules/web3-providers-http/src/index.js:110:13)
    at XMLHttpRequestEventTarget.dispatchEvent (/Users/kimdaehwan/Desktop/Code/201810756/node_modules/xhr2-cookies/dist/xml-http-request-event-target.js:34:22)
    at XMLHttpRequest._setReadyState (/Users/kimdaehwan/Desktop/Code/201810756/node_modules/xhr2-cookies/dist/xml-http-request.js:208:14)
    at XMLHttpRequest._onHttpResponseEnd (/Users/kimdaehwan/Desktop/Code/201810756/node_modul

> 과제 풀이 후 의문점 :\
이 컨트랙에서 컨트랙 잔고는 무슨 역할을 하는 것인지 정확히 모르겠습니다. 컨트랙 잔고의 변화가 없을 뿐더러 게임 진행 상황에서는 컨트랙 잔고의 사용이나 변화가 발생할 여지도 없는 것 같습니다.